### TODO: add to the requirements.txt

!pip install tweepy==3.10.0 

!pip install requests 

!pip install twython==3.9.1

In [104]:
import json
import twython
import os

In [94]:
with open("bearer_token.txt") as tokens:
    tokens_json = json.loads(tokens.read())
    #os.environ['BEARER_TOKEN'] = token.read()[:-1]
    os.environ['CONSUMER_KEY'] = tokens_json['consumer_key']
    os.environ['CONSUMER_SECRET'] = tokens_json['consumer_secret']
    os.environ['ACESS_TOKEN'] = tokens_json['access_token']
    os.environ['ACESS_TOKEN_SECRET'] = tokens_json['access_token_secret']

#### https://github.com/twitterdev/Twitter-API-v2-sample-code (no funciona)

In [96]:
consumer_key = os.environ.get("CONSUMER_KEY")
consumer_secret = os.environ.get("CONSUMER_SECRET")
access_token = os.environ.get("ACCESS_TOKEN")
access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")

In [102]:
twitter = twython.Twython(consumer_key, consumer_secret, access_token, access_token_secret) 

In [103]:
query = 'from:RafaelNadal lang:en -is:retweet -is:reply -is:quote'

In [ ]:
response = twitter.search(q=query, count=1)

In [ ]:
response